# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


☁️ Colab rilevato.
Mounted at /content/drive
Cloning into 'MistakeDetection'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 308 (delta 41), reused 72 (delta 22), pack-reused 215 (from 1)
Receiving objects: 100% (308/308), 49.98 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (122/122), done.


# Dataset Setup

In [2]:
from utils import setup_project

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)

# Ora puoi passare agli import del modello
from dataset.capitain_cook_4d_dataset import CaptainCook4D_Dataset
from models.BaselineV1_MLP import BaselineV1_MLP

# Import wandb
import wandb

Setup Progetto in: /content/MistakeDetection
source_path: /content/drive/MyDrive/MistakeDetection
Setup Dati da: /content/drive/MyDrive/MistakeDetection
Inizio setup dati...
   Sorgente: /content/drive/MyDrive/MistakeDetection
   Destinazione: /content/MistakeDetection/data
Estrazione ZIP: omnivore.zip...
Copia cartella: annotation_json...
Estrazione ZIP: omnivore_test.zip...
✅ Setup completato! Dati pronti in: /content/MistakeDetection/data


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s339450 (s339450-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB Logged in.
Device: cuda


# Dataset Split

In [3]:
import os
from dataset.capitain_cook_4d_dataset import CaptainCook4D_Dataset, DatasetSource
from dataset.utils import get_loaders

try:
    full_dataset = CaptainCook4D_Dataset(dataset_source=DatasetSource.OMNIVORE, root_dir=ROOT_DIR)
    train_loader, val_loader, test_loader = get_loaders(
        full_dataset,
        batch_size=512,
        seed=42
    )

except Exception as e:
    print(f"❌ Errore: {e}")

Loading from: /content/MistakeDetection/data/omnivore...

DATASET INFO
   Shape: torch.Size([340320, 1024]) -> 340320 Campioni, 1024 Features
FULL DATASET       | Tot: 340320 | OK: 257978 (75.8%) | ERR: 82342 (24.2%) | Ratio: 1:3.1
-----------------------------------------------------------------
TRAIN SET          | Tot: 238224 | OK: 180594 (75.8%) | ERR: 57630 (24.2%) | Ratio: 1:3.1
VALIDATION SET     | Tot: 34032  | OK: 25725 (75.6%) | ERR: 8307  (24.4%) | Ratio: 1:3.1
TEST SET           | Tot: 68064  | OK: 51659 (75.9%) | ERR: 16405 (24.1%) | Ratio: 1:3.1



In [4]:
from utils.inspect_npz import inspect_npz_from_dataset

dataset_folder = "omnivore"
npz_filename = "1_7_360p.mp4_1s_1s.npz"

# Ispezione del file .npz
inspect_npz_from_dataset(full_dataset.features_dir(), npz_filename, n_rows=3)

File: /content/MistakeDetection/data/omnivore/1_7_360p.mp4_1s_1s.npz
Chiavi presenti nel file: ['arr_0']

Array 'arr_0' - shape: (604, 1024), dtype: float32
[[ 0.6910985   0.09298898 -0.6608225  ... -0.75679165  1.2401273
  -0.5683658 ]
 [ 0.40254688 -0.4466254  -0.8645446  ... -1.2709565   0.7917245
  -0.5052321 ]
 [ 0.643613   -0.48683766 -0.88651866 ... -1.0358062   0.658605
  -0.27201462]]


In [8]:
# Configurazione esperimento
config = {
    "architecture": "BaselineV1_MLP",
    "dataset": "CaptainCook4D",
    "feature_extractor": "Omnivore",
    "input_dim": 1024,
    "batch_size": 512,
    "learning_rate": 0.0001,
    "epochs": 100,
    "pos_weight": 1.5,
    "optimizer": "Adam",
    "loss_function": "BCEWithLogitsLoss",
    "seed": 42
}

# Inizializzazione W&B
run = wandb.init(
    project="mistake-detection",
    name="baseline-mlp-v1",
    config=config,
    tags=["baseline", "mlp", "omnivore"],
    notes="Baseline MLP with Omnivore features for mistake detection"
)

print(f"🚀 W&B Run: {run.name} (ID: {run.id})")

🚀 W&B Run: baseline-mlp-v1 (ID: xc75lngy)


# W&B Configuration

# MLP (Version 1)

In [9]:
import torch
import torch.nn as nn

model = BaselineV1_MLP(1024).to(device)

# Watch del modello per tracciare gradienti e parametri
wandb.watch(model, log="all", log_freq=10)

In [10]:
lr = config["learning_rate"]
optimizer = torch.optim.Adam(model.parameters(), lr)

# Quanto pesa la classe "positiva" = classe "1" = classe "error":
# - CASO 1: rapporto effettivo del dataset
#train_pos_weight = train_cnt_0 / train_cnt_1

# - CASO 2: rapporto usato nel paper
train_pos_weight = config["pos_weight"]

print(f"Peso classe positiva: {train_pos_weight}")
train_pos_weight = torch.tensor([train_pos_weight], device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=train_pos_weight)

epochs = config["epochs"]

Peso classe positiva: 1.5


In [11]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

best_f1 = 0.0

for epoch in range(epochs):

    # -------------------------
    #        TRAIN
    # -------------------------
    model.train()
    total_loss = 0
    train_preds_list = []
    train_targets_list = []

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs)            # [B, 1]
        outputs = outputs.squeeze(1)       # [B]

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Raccolta predizioni per metriche di train
        with torch.no_grad():
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).long()
            train_preds_list.append(preds.cpu())
            train_targets_list.append(labels.cpu())

    avg_train_loss = total_loss / len(train_loader)

    # Metriche di training
    train_preds = torch.cat(train_preds_list).numpy()
    train_targets = torch.cat(train_targets_list).numpy()
    train_acc = accuracy_score(train_targets, train_preds)
    train_f1 = f1_score(train_targets, train_preds, zero_division=0)
    train_precision = precision_score(train_targets, train_preds, zero_division=0)
    train_recall = recall_score(train_targets, train_preds, zero_division=0)

    # -------------------------
    #        EVAL
    # -------------------------
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs).squeeze(1)  # logits

            # same loss as train
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

            # convert logits → probabilities → binary predictions
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).long()

            all_preds.append(preds.cpu())
            all_targets.append(labels.cpu())
            all_probs.append(probs.cpu())

    # concat
    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_probs = torch.cat(all_probs).numpy()

    avg_val_loss = total_val_loss / len(test_loader)
    val_acc = accuracy_score(all_targets, all_preds)
    val_f1 = f1_score(all_targets, all_preds, zero_division=0)
    val_precision = precision_score(all_targets, all_preds, zero_division=0)
    val_recall = recall_score(all_targets, all_preds, zero_division=0)

    # Confusion Matrix
    cm = confusion_matrix(all_targets, all_preds)

    # Log su W&B
    wandb.log({
        # Training metrics
        "train/loss": avg_train_loss,
        "train/accuracy": train_acc,
        "train/f1": train_f1,
        "train/precision": train_precision,
        "train/recall": train_recall,

        # Validation metrics
        "val/loss": avg_val_loss,
        "val/accuracy": val_acc,
        "val/f1": val_f1,
        "val/precision": val_precision,
        "val/recall": val_recall,

        # Confusion Matrix
        "val/confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=all_targets,
            preds=all_preds,
            class_names=["No Error", "Error"]
        ),

        # Learning
        "learning_rate": optimizer.param_groups[0]['lr'],
        "epoch": epoch + 1
    })

    print(f"Epoch {epoch+1}/{epochs} "
          f"- Train Loss: {avg_train_loss:.4f} "
          f"- Val Loss: {avg_val_loss:.4f} "
          f"- Val Acc: {val_acc:.4f} "
          f"- Val F1: {val_f1:.4f} "
          f"- Val Precision: {val_precision:.4f} "
          f"- Val Recall: {val_recall:.4f}")

    # Salva il miglior modello
    if val_f1 > best_f1:
        best_f1 = val_f1
        checkpoint_path = os.path.join(ROOT_DIR, "checkpoints", f"best_model_f1_{best_f1:.4f}.pth")
        os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': val_f1,
            'val_acc': val_acc,
        }, checkpoint_path)

        # Salva come artifact su W&B
        artifact = wandb.Artifact(
            name=f"model-{run.id}",
            type="model",
            description=f"Best model with F1={best_f1:.4f}",
            metadata={
                "epoch": epoch + 1,
                "val_f1": val_f1,
                "val_acc": val_acc,
                "architecture": config["architecture"]
            }
        )
        artifact.add_file(checkpoint_path)
        wandb.log_artifact(artifact)

        print(f"✅ Nuovo miglior modello salvato! F1: {best_f1:.4f}")

print("\n🎉 Training completato!")
print(f"Miglior F1 Score: {best_f1:.4f}")

Epoch 1/100 - Train Loss: 0.6735 - Val Loss: 0.6474 - Val Acc: 0.7684 - Val F1: 0.2383 - Val Precision: 0.5751 - Val Recall: 0.1503
✅ Nuovo miglior modello salvato! F1: 0.2383
Epoch 2/100 - Train Loss: 0.6373 - Val Loss: 0.6205 - Val Acc: 0.7793 - Val F1: 0.3694 - Val Precision: 0.5932 - Val Recall: 0.2682
✅ Nuovo miglior modello salvato! F1: 0.3694
Epoch 3/100 - Train Loss: 0.6115 - Val Loss: 0.5968 - Val Acc: 0.7888 - Val F1: 0.3955 - Val Precision: 0.6376 - Val Recall: 0.2867
✅ Nuovo miglior modello salvato! F1: 0.3955
Epoch 4/100 - Train Loss: 0.5897 - Val Loss: 0.5765 - Val Acc: 0.7960 - Val F1: 0.4406 - Val Precision: 0.6500 - Val Recall: 0.3333
✅ Nuovo miglior modello salvato! F1: 0.4406
Epoch 5/100 - Train Loss: 0.5698 - Val Loss: 0.5597 - Val Acc: 0.8029 - Val F1: 0.4815 - Val Precision: 0.6581 - Val Recall: 0.3796
✅ Nuovo miglior modello salvato! F1: 0.4815
Epoch 6/100 - Train Loss: 0.5547 - Val Loss: 0.5466 - Val Acc: 0.8084 - Val F1: 0.4911 - Val Precision: 0.6824 - Val Rec

# Results & Visualization

In [14]:
# Log della tabella con esempi di predizioni
model.eval()
predictions_table = wandb.Table(
    columns=["ID", "True Label", "Predicted", "Probability", "Correct"]
)

global_id = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs).squeeze(1)
        probs = torch.sigmoid(outputs)
        preds = (probs >= 0.5).long()

        for i in range(min(50, len(labels))):
            predictions_table.add_data(
                global_id,
                labels[i].item(),
                preds[i].item(),
                probs[i].item(),
                preds[i].item() == labels[i].item()
            )
            global_id += 1

        break  # solo un batch

wandb.log({"predictions/sample_table": predictions_table})

# Summary
wandb.run.summary["best_f1"] = best_f1
wandb.run.summary["final_val_acc"] = val_acc
wandb.run.summary["final_val_precision"] = val_precision
wandb.run.summary["final_val_recall"] = val_recall

print("✅ Tabella predizioni e summary caricati su W&B")

✅ Tabella predizioni e summary caricati su W&B


In [15]:
# Chiudi il run di W&B
wandb.finish()
print("🏁 W&B run terminato")

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,▁▂▃▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████████
train/f1,▁▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
train/loss,█▇▆▆▆▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/precision,▁▂▂▂▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
train/recall,▁▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████
val/accuracy,▁▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
val/f1,▁▂▃▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
val/loss,█▆▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+2,...


🏁 W&B run terminato
